In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense
import keras
from tensorflow.keras.layers import Dense, Layer
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape

In [ ]:
(training_images, training_labels), (test_images, test_labels) = datasets.mnist.load_data()
print("training_images.shape:",training_images.shape)
print("test_images.shape:",test_images.shape)

In [ ]:
# 將後門圖形塞到圖片中
def install_backdoor_image(images) :
    images[25][26] = 25
    images[26][25] = 25
    images[26][26] = 25
    images[26][27] = 25
    images[27][26] = 25
    

backdoor_images = training_images.copy()
for i in backdoor_images :
    install_backdoor_image(i)

training_images = np.append(backdoor_images,training_images,axis=0)

backdoor_labels = np.ones(training_labels.shape[0])
training_labels = np.zeros(training_labels.shape[0])
training_labels = np.append(backdoor_labels,training_labels,axis=0)


backdoor_test_images = test_images.copy()
for i in backdoor_test_images :
    install_backdoor_image(i)
    
test_images = np.append(backdoor_test_images,test_images,axis=0)

test_backdoor_labels = np.ones(test_labels.shape[0])
test_labels = np.zeros(test_labels.shape[0])
test_labels = np.append(test_backdoor_labels,test_labels,axis=0)


In [ ]:
# 顯示一下有打後門跟沒打後門的差異
fig = plt.figure(figsize=(28, 28))

fig.add_subplot(1,2,1)
plt.imshow(training_images[0],cmap='gray')

fig.add_subplot(1,2,2)
plt.imshow(training_images[60001],cmap='gray')

plt.show()

In [ ]:
# 把後面二維的部分攤平成一維
t_training_images = training_images.reshape(120000, 784)
t_test_images = test_images.reshape(20000, 784)

#轉換格式為 float32
t_training_images = t_training_images.astype('float32')
t_test_images = t_test_images.astype('float32')

# 將數值做正規化
t_training_images  = t_training_images / 255.0
t_test_images = t_test_images / 255.0


t_training_labels = training_labels.astype('float32')
t_test_labels = test_labels.astype('float32')



In [ ]:
# 建立屬於自己的 model
model = Sequential(name='attack_model')
model.add(Dense(128, input_dim=784, activation=tf.nn.relu, name='attack_1'))
model.add(Dense(2,activation=tf.nn.softmax, name='attack_2'))

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = tf.keras.losses.sparse_categorical_crossentropy ,
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(t_training_images, t_training_labels, epochs=5, batch_size=128)

In [ ]:
loss, acc = model.evaluate(t_test_images, t_test_labels, batch_size=128)
print("loss rate:", loss, " accuracy: " , (100.0 * acc))

In [ ]:
model.save('mnist_backdoor_model.h5',save_format='h5')

In [ ]:
plt.imshow(test_images[0],cmap='gray')

In [ ]:
model.predict(t_test_images[0].reshape(1,784))

In [ ]:
model.predict(t_test_images[10001].reshape(1,784))

In [ ]:
plt.imshow(test_images[10001],cmap='gray')